- [Embedding - numbering (one-hot-encoding)](#Embedding---numbering-(one-hot-encoding))

In [3]:
import numpy as np
import pandas as pd

# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from gensim.models import CoherenceModel

import khaiii
import sentencepiece
from khaiii import KhaiiiApi

from gensim.models import fasttext

# import pyLDAvis
# import pyLDAvis.gensim
import matplotlib.pyplot as plt

import re
import pickle
import tool
from tqdm import tqdm
from pprint import pprint

In [2]:
song = pd.read_json("res/song_meta.json")[["id", "song_name", "artist_id_basket", "artist_name_basket",\
                                            "album_id", "album_name", "song_gn_gnr_basket",\
                                            "song_gn_dtl_gnr_basket", "issue_date"]]
genre = pd.read_json("res/genre_gn_all.json", typ="series")
train = pd.read_json("res/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val = pd.read_json("res/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
test = pd.read_json("res/test.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
song.name = "song"
genre.name = "genre"
train.name = "train"
val.name = "val"
test.name = "test"
display(song, genre, train, val, test)

,id,song_name,artist_id_basket,artist_name_basket,album_id,album_name,song_gn_gnr_basket,song_gn_dtl_gnr_basket,issue_date
0,0,Feelings,[2727],[Various Artists],2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,[GN0900],[GN0901],20140512
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],[Murray Perahia],376431,"Bach : Partitas Nos. 2, 3 & 4",[GN1600],"[GN1601, GN1606]",20080421
2,2,Solsbury Hill (Remastered 2002),[3361],[Peter Gabriel],4698747,Hit,[GN0900],[GN0901],20180518
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[838543],[Matoma],2644882,Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],"[GN1102, GN1101]",20151016
4,4,그남자 그여자,[560160],[Jude Law],2008470,그남자 그여자,[GN1800],"[GN1802, GN1801]",20110824
...,...,...,...,...,...,...,...,...,...
707984,707984,Coffin For Head Of State,[166499],[Fela Kuti],65254,The Best Best Of The Black President,[GN2000],[GN2001],19991219
707985,707985,Change Of Heart,[11837],[Cyndi Lauper],44141,True Colors,[GN0900],[GN0901],19860000
707986,707986,스치듯 안녕,[437],[윤종신],2662866,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,[GN0100],"[GN0105, GN0101]",20160120
707987,707987,숲의 빛,[729868],[Nature Piano],2221722,명상의 시간을 위한 뉴에이지 음악,[GN1800],"[GN1807, GN1801]",20131217


GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Name: genre, Length: 254, dtype: object

,id,plylst_title,songs,tags,like_cnt,updt_date
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",17,2017-08-28 07:09:34.000
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",33,2019-12-05 15:15:18.000
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",3,2020-04-17 04:31:11.000
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",[일렉],13,2015-12-24 17:23:19.000
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",4,2019-08-16 20:59:22.000
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",4,2019-10-25 23:40:42.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],1675,2019-05-27 14:14:33.000
1,131447,앨리스테이블,[],[],1,2014-07-16 15:24:24.000
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],62,2008-06-21 23:26:22.000
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],20,2017-10-30 18:15:43.000
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[잔잔한],17,2015-12-17 14:06:05.000
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",10,2020-04-16 21:35:44.000
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[],0,2019-03-27 15:27:40.000
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[],31,2015-11-18 11:49:09.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[],6,2012-09-29 01:57:26.000
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[나만의Best3, 인디아티스트들의추천음악]",0,2019-12-17 14:06:45.000
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브],21,2015-05-23 10:44:48.000
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[분위기],0,2019-03-14 09:47:34.000
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[],38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[추억],16,2016-04-18 11:02:09.000
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[띵곡의, 우울, 분위기, 드라이브, 산책]",1,2020-04-08 07:15:59.000
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",[기분전환],6,2016-06-29 00:57:21.000
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[여름],4,2015-06-06 09:52:01.000


In [8]:
train80 = pd.read_json("arena_data/orig/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val20q = pd.read_json("arena_data/questions/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val20a = pd.read_json("arena_data/answers/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
display(train80, val20q, val20a)

,id,plylst_title,songs,tags,like_cnt,updt_date
0,147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...","[힐링, 휴식, 밤, 새벽]",12,2016-06-23 10:06:27.000
1,50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",[팝],0,2013-08-15 13:17:11.000
2,116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...",[뉴에이지],23,2015-09-03 16:51:50.000
3,55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...","[하드락, 록스피릿, 댄스]",1,2017-01-09 15:41:25.000
4,125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...","[힐링, 휴식, 기분전환]",715,2016-02-22 12:32:50.000
...,...,...,...,...,...,...
92051,149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...","[90년생, 회상, 추억, 좋은노래, 80년생, 옛날노래]",155,2020-01-15 15:15:45.000
92052,35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",[팝],8,2010-03-23 00:03:00.000
92053,59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...","[여행, 발라드, 기분전환, 사랑]",3,2019-05-15 13:26:07.000
92054,9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...","[소울, 알앤비]",51,2013-12-24 14:40:01.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,119064,재즈! 신나는재즈!,[],[],18,2017-05-03 17:18:07.000
1,64891,"가을의 문턱에서! anywhere, anybody :) 듣기좋은 음악","[274165, 552540, 104677, 188486, 271300, 25864...",[드라이브],29,2019-10-17 09:32:41.000
2,68628,ⓗ 감성가요,"[275414, 477905, 256688, 276685, 109237, 54408...",[랩],0,2013-12-03 11:51:26.000
3,100993,위로의 형태,[],[],2,2018-03-12 11:23:21.000
4,10417,"조금 지친 밤, 눈 감고 듣고 싶은 위로의 음악","[632115, 307581, 357367, 112832, 648635, 60663...",[],612,2019-03-06 18:35:20.000
...,...,...,...,...,...,...
23010,115244,밤하늘 보며 듣고 싶은 잔잔한 음악,"[222048, 562040, 295682, 474983, 405990, 61620...",[잔잔한],6,2016-09-15 12:05:43.000
23011,124493,MBC FM4U 라디오 14.12.01 ~ 31 : 그 달의 음악,"[158590, 375777, 598734, 63734, 112399, 76888,...",[힐링],7,2015-01-01 11:13:45.000
23012,65279,한겨울의 따뜻한 핫초코 같은 음악,[],[],38,2011-01-06 00:04:33.000
23013,53828,우리 함께 했던 시간들이 담긴 음악,"[353960, 221785, 234462, 22976, 31784, 403481,...",[],0,2015-05-05 13:29:48.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,119064,재즈! 신나는재즈!,"[231820, 13676, 593705, 23746, 429366, 376182,...",[재즈],18,2017-05-03 17:18:07.000
1,64891,"가을의 문턱에서! anywhere, anybody :) 듣기좋은 음악","[448698, 156706, 185733, 117595, 497066, 28628...",[가을],29,2019-10-17 09:32:41.000
2,68628,ⓗ 감성가요,"[407073, 529965, 700041, 157285, 578509, 65827...",[힙합],0,2013-12-03 11:51:26.000
3,100993,위로의 형태,"[445568, 114172, 234233, 348174, 521628, 43010...",[슬픔],2,2018-03-12 11:23:21.000
4,10417,"조금 지친 밤, 눈 감고 듣고 싶은 위로의 음악","[468023, 637294, 130632, 340538, 290477, 39356...","[위로, 혼자있고싶을때, 따뜻한]",612,2019-03-06 18:35:20.000
...,...,...,...,...,...,...
23010,115244,밤하늘 보며 듣고 싶은 잔잔한 음악,"[335474, 105039, 401182, 398584, 311731, 51024...",[까페],6,2016-09-15 12:05:43.000
23011,124493,MBC FM4U 라디오 14.12.01 ~ 31 : 그 달의 음악,"[370002, 551409, 95323, 346969, 579728, 60117,...","[휴식, 잔잔한]",7,2015-01-01 11:13:45.000
23012,65279,한겨울의 따뜻한 핫초코 같은 음악,"[572238, 174778, 198528, 280899, 310653, 41008...",[팝],38,2011-01-06 00:04:33.000
23013,53828,우리 함께 했던 시간들이 담긴 음악,"[315273, 245317, 359517, 593278, 594982, 22588...",[발라드],0,2015-05-05 13:29:48.000


### Separate Korean and English from *playlist-title* and *tags*

In [3]:
%%time
# return type : string
run = False
data = train
dtype = "str"
if run:
    ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dense=True, limit=None)
    with open(f'bin/{dtype}_{data.name}.p', 'wb') as f:
        _tuple = (ko_title, ko_tag, en_title, en_tag)
        pickle.dump(_tuple, f)
if not run: # data load
    with open(f'bin/{dtype}_{data.name}.p', 'rb') as f:
        ko_title, ko_tag, en_title, en_tag = pickle.load(f)
ko_title, ko_tag, en_title, en_tag

CPU times: user 88.4 ms, sys: 17 ms, total: 105 ms
Wall time: 108 ms


({0: '여행같은 음악',
  1: '요즘 너 말야',
  2: '편하게  잔잔하게 들을 수 있는 곡',
  3: '크리스마스 분위기에 흠뻑 취하고 싶을때',
  4: '추억의 노래 ㅋ',
  6: '짝사랑 고백 사랑 이별 슬픔  감성을 자극하는곡들',
  7: '멍청이  내맘도 몰라',
  9: '록 메탈      이일우',
  10: '걸그룹 땐쓰쏭',
  11: '노래로 의지를 불태우자   일 다짐   용 프로필뮤직',
  12: '지친 너를 위로해줄 제목편지 준비해봤어',
  13: '트렌디하고 그루브한 힙합 알앤비',
  15: '걍게임할때듣는음악',
  16: '나만 알고싶은 노래들',
  17: '축가듀엣',
  18: '이렇게 비 내리는 날이면  너도 내 생각 할까',
  19: '올림픽 스밍 목록',
  20: '조용히 맥주 한잔 할때 새벽감성',
  22: '추억의 명화를 만나는 시간  영화   모음',
  23: '기분좋은 여름날',
  24: '지친 하루 끝  힐링이 필요한 당신에게 추천하는 인디곡',
  25: '새벽 찬바람이 침대위를 감쌀때 국외',
  26: '기분좋은 햇살같은 숨겨진 명곡들',
  27: '사랑스러운 남녀듀엣',
  28: '즐겨듣는소래',
  29: '닥힙추  월',
  30: '거슈윈에서 스타워즈까지  미국 대표 클래식',
  31: '겨울이 기다리며 미리 들어요',
  32: '아침이 상쾌해지는 빅밴드의 스윙스윙',
  33: '여행가는 길 이런음악과 함께 고고씽',
  34: '믿고듣는 하우스 일렉트로니카',
  35: '프로포즈 음악',
  36: '보아일본곡',
  37: '안다르 레깅스 입고 운동 시작하자',
  38: '발라드로 감성을 발라드림 슬픔주의',
  39: '크리스마스를 책임질 캐롤 모음',
  40: '아무것도 위로 되지 못할 때 위로가 되어준 음악',
  41: '트렌디한',
  42: '년대 우리나라 라디오에서 흘러나오던 명곡들',
  43: '영화   를 만나다',
  44: '우울

In [10]:
# return type : list
run = False
data = train
if run:
    ko_title_lst, ko_tag_lst, en_title_lst, en_tag_lst = \
    tool.extract_playlist_from(data, dtype="list", dense=True, limit=None)
    with open(f'bin/list_{data.name}.p', 'wb') as f:
        _tuple = (ko_title_lst, ko_tag_lst, en_title_lst, en_tag_lst)
        pickle.dump(_tuple, f)
else:
    with open(f'bin/list_{data.name}.p', 'rb') as f:
        ko_title_lst, ko_tag_lst, en_title_lst, en_tag_lst = pickle.load(f)
ko_title_lst, ko_tag_lst, en_title_lst, en_tag_lst

({0: ['여행같은', '음악'],
  1: ['요즘', '너', '말야'],
  2: ['편하게', '잔잔하게', '들을', '수', '있는', '곡'],
  3: ['크리스마스', '분위기에', '흠뻑', '취하고', '싶을때'],
  4: ['추억의', '노래', 'ㅋ'],
  6: ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성을', '자극하는곡들'],
  7: ['멍청이', '내맘도', '몰라'],
  9: ['록', '메탈', '이일우'],
  10: ['걸그룹', '땐쓰쏭'],
  11: ['노래로', '의지를', '불태우자', '일', '다짐', '용', '프로필뮤직'],
  12: ['지친', '너를', '위로해줄', '제목편지', '준비해봤어'],
  13: ['트렌디하고', '그루브한', '힙합', '알앤비'],
  15: ['걍게임할때듣는음악'],
  16: ['나만', '알고싶은', '노래들'],
  17: ['축가듀엣'],
  18: ['이렇게', '비', '내리는', '날이면', '너도', '내', '생각', '할까'],
  19: ['올림픽', '스밍', '목록'],
  20: ['조용히', '맥주', '한잔', '할때', '새벽감성'],
  22: ['추억의', '명화를', '만나는', '시간', '영화', '모음'],
  23: ['기분좋은', '여름날'],
  24: ['지친', '하루', '끝', '힐링이', '필요한', '당신에게', '추천하는', '인디곡'],
  25: ['새벽', '찬바람이', '침대위를', '감쌀때', '국외'],
  26: ['기분좋은', '햇살같은', '숨겨진', '명곡들'],
  27: ['사랑스러운', '남녀듀엣'],
  28: ['즐겨듣는소래'],
  29: ['닥힙추', '월'],
  30: ['거슈윈에서', '스타워즈까지', '미국', '대표', '클래식'],
  31: ['겨울이', '기다리며', '미리', '들어요'],
  32: ['아침이', '상쾌해지는', '빅밴

### Separate Korean and English from *song-name*, *name-of-artists* and *album-name*

In [5]:
%%time
# return type : string
run = False
dtype = "str"
if run:
    ko_name, ko_album, ko_artist, en_name, en_album, en_artist = tool.extract_song_from(song, dtype=dtype, limit=None)
    with open(f'bin/{dtype}_song.p', 'wb') as f:
        _tuple = (ko_name, ko_album, ko_artist, en_name, en_album, en_artist)
        pickle.dump(_tuple, f)
if not run: # data load
    with open(f'bin/{dtype}_song.p', 'rb') as f:
        ko_name, ko_album, ko_artist, en_name, en_album, en_artist = pickle.load(f)
ko_name, ko_album, ko_artist, en_name, en_album, en_artist

CPU times: user 519 ms, sys: 125 ms, total: 644 ms
Wall time: 650 ms


({4: '그남자 그여자',
  6: '시벨리우스   슬픈 왈츠 작품번호',
  9: '사랑  그대라는 멜로디',
  18: '숙면과 휴식에 좋은 편안한 빗소리  백색소음',
  19: '무얼 기다리나   조원선',
  20: '길병민  유채훈  한기주   팬텀싱어   유채훈 가창곡',
  25: '뒤돌아봐',
  29: '창밖에 잠수교가 보인다',
  32: '먼지',
  35: '찾고 있니',
  36: '연가',
  48: '나의 노래',
  49: '야상곡',
  51: '초콜렛 먹는 소리  고요함',
  53: '내가 그리스도와 함께',
  54: '있어줄래',
  58: '국화꽃 향기   김세찬',
  59: '고향의 봄',
  60: '논현     최재문',
  62: '잠시 길을 잃다   신보경',
  65: '차이콥스키   잠자는 숲속의 미녀 작품번호      번  파노라마',
  66: '그리움으로 물든 노을',
  67: '떠나지 못해',
  68: '나를 위한 휴식',
  70: '너에게 전하지 못한 말',
  81: '두 번째 인생',
  85: '매미의 꿈           당신을 위로하고 싶은 마음에서 나온 가벼운 성의',
  86: '우리 사랑 이야기',
  92: '참쓰다',
  96: '가르마',
  97: '따뜻한 바람 부는 계절',
  99: '장일남   비목',
  105: '하늘은 말없이',
  111: '교생쌤',
  112: '예수를 나의 구주삼고',
  115: '마음의 소리',
  124: '드뷔시   베르가마스크 모음곡    번  달빛   명상음악 클래식',
  127: '걱정말아요 그대',
  130: '하이든   현악  중주  번 바장조 작품번호      악장',
  131: '보이비',
  133: '충무로의 밤',
  135: '그런 사람',
  136: '나 주를 멀리 떠났다',
  143: '커피 한잔 할래요',
  146: '러빙러빙',
  148: '어디쯤 있나요',
  149: '겨울 애상',
 

In [6]:
# return type : list
run = False
if run:
    ko_name_lst, ko_album_lst, ko_artist_lst, en_name_lst, en_album_lst, en_artist_lst = \
    tool.extract_song_from(song, dtype="list", dense=True, limit=None)
    with open(f'bin/list_song.p', 'wb') as f:
        _tuple = (ko_name_lst, ko_album_lst, ko_artist_lst, en_name_lst, en_album_lst, en_artist_lst)
        pickle.dump(_tuple, f)
else:
    with open(f'bin/list_song.p', 'rb') as f:
        ko_name_lst, ko_album_lst, ko_artist_lst, en_name_lst, en_album_lst, en_artist_lst = pickle.load(f)
ko_name_lst, ko_album_lst, ko_artist_lst, en_name_lst, en_album_lst, en_artist_lst

({4: ['그남자', '그여자'],
  6: ['시벨리우스', '슬픈', '왈츠', '작품번호'],
  9: ['사랑', '그대라는', '멜로디'],
  18: ['숙면과', '휴식에', '좋은', '편안한', '빗소리', '백색소음'],
  19: ['무얼', '기다리나', '조원선'],
  20: ['길병민', '유채훈', '한기주', '팬텀싱어', '유채훈', '가창곡'],
  25: ['뒤돌아봐'],
  29: ['창밖에', '잠수교가', '보인다'],
  32: ['먼지'],
  35: ['찾고', '있니'],
  36: ['연가'],
  48: ['나의', '노래'],
  49: ['야상곡'],
  51: ['초콜렛', '먹는', '소리', '고요함'],
  53: ['내가', '그리스도와', '함께'],
  54: ['있어줄래'],
  58: ['국화꽃', '향기', '김세찬'],
  59: ['고향의', '봄'],
  60: ['논현', '최재문'],
  62: ['잠시', '길을', '잃다', '신보경'],
  65: ['차이콥스키', '잠자는', '숲속의', '미녀', '작품번호', '번', '파노라마'],
  66: ['그리움으로', '물든', '노을'],
  67: ['떠나지', '못해'],
  68: ['나를', '위한', '휴식'],
  70: ['너에게', '전하지', '못한', '말'],
  81: ['두', '번째', '인생'],
  85: ['매미의', '꿈', '당신을', '위로하고', '싶은', '마음에서', '나온', '가벼운', '성의'],
  86: ['우리', '사랑', '이야기'],
  92: ['참쓰다'],
  96: ['가르마'],
  97: ['따뜻한', '바람', '부는', '계절'],
  99: ['장일남', '비목'],
  105: ['하늘은', '말없이'],
  111: ['교생쌤'],
  112: ['예수를', '나의', '구주삼고'],
  115: ['마음의', '소리'],
  124: ['드뷔시'

### Tokenization

In [11]:
run = False
if run:
    api = KhaiiiApi()
    ko_nouns = {}
    # for data in [ko_title, ko_tag, ko_name, ko_album, ko_artist]:
    ko_nouns["title"] = tool.ko_extract_tag_from(ko_title, api, limit=None)
    ko_nouns["tag"] = tool.ko_extract_tag_from(ko_tag, api)
    # ko_nouns["name"] = tool.ko_extract_tag_from(ko_name, api)
    # ko_nouns["album"] = tool.ko_extract_tag_from(ko_album)
    # ko_nouns["artist"] = tool.ko_extract_tag_from(ko_artist)
    with open(f'bin/ko_nouns.p', 'wb') as f:
        pickle.dump(ko_nouns, f)
else:
    with open(f'bin/ko_nouns.p', 'rb') as f:
        ko_nouns = pickle.load(f)
ko_nouns

{'title': {0: ['여행', '음악'],
  1: ['요즘', '말'],
  2: ['곡'],
  3: ['크리스마스', '분위기', '때'],
  4: ['추억', '노래', 'ㅋ'],
  6: ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '자극', '곡'],
  7: ['멍청', '맘'],
  9: ['메'],
  10: ['때', '쓰쏭'],
  11: ['노래', '의지', '일', '다짐', '용', '프로', '필뮤직'],
  12: ['위로', '제목', '편지', '준비'],
  13: ['트렌디', '힙합'],
  15: ['임', '음악'],
  16: ['노래'],
  17: ['축가듀엣'],
  18: ['비', '날', '생각'],
  19: ['올림픽', '스밍', '목록'],
  20: ['맥주', '한잔', '때', '새벽', '감성'],
  22: ['추억', '명화', '시간', '영화', '모음'],
  23: ['기분', '여름날'],
  24: ['하루', '끝', '링', '필요', '추천', '인디곡'],
  25: ['새벽', '찬바람', '침대위', '감', '때', '국외'],
  26: ['기분', '햇살', '명곡'],
  27: ['사랑', '남녀', '듀엣'],
  28: ['소래'],
  29: ['닥힙추', '월'],
  30: ['거', '워', '대표', '클래식'],
  31: ['겨울'],
  32: ['아침', '밴', '스윙스윙'],
  33: ['여행가', '길', '악', '고고씽'],
  34: ['하우스', '로'],
  35: ['프로', '즈', '음악'],
  37: ['레깅', '운동', '시작'],
  38: ['발', '드', '감성', '발', '드림', '슬픔주의'],
  39: ['크리스마스', '질', '모음'],
  40: ['아무것', '위', '때', '위로', '음악'],
  41: ['트렌디'],
  42: ['우리나라', '라

In [13]:
ko_nouns["tag"]

{1: ['추억', '회상'],
 2: ['까페'],
 3: ['연말',
  '눈',
  '날',
  '캐',
  '분위기',
  '크리스마스캐럴',
  '겨울',
  '노래',
  '크리스마스',
  '겨울왕국',
  '크리스마스'],
 4: ['댄스'],
 5: ['운동', '드라이브', '트로피컬', '우스', '기분', '전환', '팝', '트렌드'],
 6: ['짝사랑', '취향', '저격', '슬픔', '고백', '사랑', '이별'],
 7: ['추억', '회상'],
 8: ['로', '포크', '메탈', '락', '댄스', '인디'],
 9: ['에센셜', '메탈', '락'],
 10: ['댄스', '걸그룹', '댄스', '스트레스', '해'],
 11: ['새해',
  '여행',
  '프로',
  '필음악',
  '카카오톡',
  '기분',
  '전환',
  '소원',
  '프로필',
  '소망',
  '다짐',
  '카톡'],
 12: ['우울'],
 13: ['힙합', '느낌', '밤', '새벽', '감각', '드라이브', '국내'],
 14: ['가을', '재즈'],
 16: ['감성'],
 17: ['봄', '사랑'],
 18: ['비', '날', '각날때'],
 19: ['스밍', '목록', '폐막식', '올림픽', '엑소'],
 20: ['혼자', '새벽', '감', '고민', '맥주한잔'],
 21: ['카페', '재즈'],
 23: ['댄스'],
 24: ['감성', '어쿠스틱', '새벽', '편안', '인디', '밤'],
 25: ['밤', '새벽'],
 26: ['힐링', '드라이브', '에너지', '인디', '여행'],
 27: ['듀엣', '취향', '저격', '피쳐링', '사랑', '남녀'],
 28: ['까페', '사랑'],
 29: ['힙합', '신인', '국', '인디', '신'],
 30: ['거', '소디인블루', '미국클래식', '영', '클래식', '메리칸클래식'],
 31: ['겨울', '기분', '전환'],

### title : [Nouns] / tags : [Original, Nouns]

In [15]:
data = train80
api = KhaiiiApi()
ko_nouns = {}
ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dense=True)
ko_nouns["title"] = tool.ko_extract_tag_from(ko_title, api, limit=None)
ko_nouns["tag"] = tool.ko_extract_tag_from(ko_tag, api)
ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dtype="list", dense=True)

In [21]:
ko_tag, en_tag, ko_nouns["title"], ko_nouns["tag"]

({0: ['힐링', '휴식', '밤', '새벽'],
  1: ['팝'],
  2: ['뉴에이지'],
  3: ['하드락', '록스피릿', '댄스'],
  4: ['힐링', '휴식', '기분전환'],
  5: ['요즘듣는노래'],
  6: ['카페', '인디'],
  7: ['로우파이', '감성', '자기전에듣기좋은', '알앤비힙합'],
  8: ['댄스'],
  9: ['발라드'],
  10: ['휴식', '힐링', '여행', '산책'],
  11: ['밤', '까페', '새벽'],
  12: ['발라드'],
  13: ['일렉'],
  14: ['내한', '감성', '취향저격', '팝', '인디팝', '밴드', '레이니', '음색', '록'],
  15: ['카페', '피아노', '뉴에이지', '봄', '재즈'],
  16: ['슬픔', '이별', '설렘', '사랑'],
  17: ['감성', '밤', '알앤비'],
  18: ['몽롱', '명곡', '락발라드', '해외락', '기분전환', '락음악', '부드러운', '감성락'],
  19: ['히피'],
  20: ['발라드'],
  21: ['기분전환', '여행', '드라이브', '신나는'],
  22: ['아침', '운동', '드라이브', '여행', '기분전환', '스트레스', '신나는'],
  23: ['설렘', '사랑'],
  24: ['휴식', '힐링', '설렘', '사랑'],
  25: ['록메탈'],
  26: ['크리스마스', '재즈'],
  27: ['발라드'],
  28: ['잔잔한', '감성', '밤', '새벽', '취향저격', '발라드', '사랑'],
  29: ['힐링', '휴식', '밤', '새벽'],
  30: ['피아노'],
  31: ['감성', '밤', '발라드', '추억', '드라이브', '년대', '사랑', '이별'],
  32: ['슬픔', '추억', '이별', '회상'],
  33: ['힙합', '발라드', '댄스', '플레이리스트', '이것저것', '랜덤', '팝송

In [75]:
t80 = train80[['id', 'plylst_title', 'songs']]
t80["tags"] = np.nan
t80["tags"] = t80["tags"].astype('object')
display(t80)
for i in tqdm(train80.index):
    a = ko_tag[i] if i in ko_tag else []
    b = en_tag[i] if i in en_tag else []
    c = ko_nouns["title"][i] if i in ko_nouns["title"] else []
    d = ko_nouns["tag"][i] if i in ko_nouns["tag"] else []
    e = list(set(a + b + c + d))
    t80.at[i, "tags"] = e
t80

/Users/jehunyoo/anaconda3/envs/cosmos/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jehunyoo/anaconda3/envs/cosmos/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,plylst_title,songs,tags
0,147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...",NaN
1,50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",NaN
2,116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...",NaN
3,55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...",NaN
4,125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...",NaN
...,...,...,...,...
92051,149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...",NaN
92052,35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",NaN
92053,59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...",NaN
92054,9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...",NaN


,id,plylst_title,songs,tags
0,147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...","[새벽, 휴식, 힐링, 밤]"
1,50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",[팝]
2,116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...","[뉴에이, 마음, 뉴에이지, 피아노, 숙면]"
3,55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...","[곡, 울, 하드락, 댄스, 록스피릿]"
4,125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...","[전환, 음악, 휴식, 필수, 힐링, 스피커, 기분전환, 기분, 힙합]"
...,...,...,...,...
92051,149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...","[년생, 좋은노래, 노래, 회상, 추억, 년, 옛날노래, 모음]"
92052,35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",[팝]
92053,59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...","[전환, 베스트, 사랑, 추억, 발라드, 여행, 기분전환, 기분]"
92054,9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...","[선정, 소울, 알앤비]"


In [76]:
train80

,id,plylst_title,songs,tags,like_cnt,updt_date
0,147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...","[힐링, 휴식, 밤, 새벽]",12,2016-06-23 10:06:27.000
1,50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",[팝],0,2013-08-15 13:17:11.000
2,116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...",[뉴에이지],23,2015-09-03 16:51:50.000
3,55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...","[하드락, 록스피릿, 댄스]",1,2017-01-09 15:41:25.000
4,125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...","[힐링, 휴식, 기분전환]",715,2016-02-22 12:32:50.000
...,...,...,...,...,...,...
92051,149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...","[90년생, 회상, 추억, 좋은노래, 80년생, 옛날노래]",155,2020-01-15 15:15:45.000
92052,35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",[팝],8,2010-03-23 00:03:00.000
92053,59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...","[여행, 발라드, 기분전환, 사랑]",3,2019-05-15 13:26:07.000
92054,9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...","[소울, 알앤비]",51,2013-12-24 14:40:01.000


### Embedding - numbering (one-hot-encoding)

In [19]:
%%time
# numbering (Korean + English)
# ko_title_lst, ko_tag_lst, en_title_lst, en_tag_lst
# ko_name_lst, ko_album_lst, ko_artist_lst, en_name_lst, en_album_lst, en_artist_lst
run = False
if run:
    data = [ko_title_lst, ko_tag_lst, en_title_lst, en_tag_lst, \
            ko_name_lst, ko_album_lst, ko_artist_lst, en_name_lst, en_album_lst, en_artist_lst]
    words, inverse = tool.we_numbering(data, limit=None, return_inverse=True)
    with open('bin/ko_en_numbering.p', 'wb') as f:
        pickle.dump(words, f)
    with open('bin/ko_en_numbering_inverse.p', 'wb') as f:
        pickle.dump(inverse, f)
else:
    with open('bin/ko_en_numbering.p', 'rb') as f:
        words = pickle.load(f)
    with open('bin/ko_en_numbering_inverse.p', 'rb') as f:
        inverse = pickle.load(f)
words, inverse

CPU times: user 141 ms, sys: 37.4 ms, total: 179 ms
Wall time: 182 ms


({'여행같은': 0,
  '음악': 1,
  '요즘': 2,
  '너': 3,
  '말야': 4,
  '편하게': 5,
  '잔잔하게': 6,
  '들을': 7,
  '수': 8,
  '있는': 9,
  '곡': 10,
  '크리스마스': 11,
  '분위기에': 12,
  '흠뻑': 13,
  '취하고': 14,
  '싶을때': 15,
  '추억의': 16,
  '노래': 17,
  'ㅋ': 18,
  '짝사랑': 19,
  '고백': 20,
  '사랑': 21,
  '이별': 22,
  '슬픔': 23,
  '감성을': 24,
  '자극하는곡들': 25,
  '멍청이': 26,
  '내맘도': 27,
  '몰라': 28,
  '록': 29,
  '메탈': 30,
  '이일우': 31,
  '걸그룹': 32,
  '땐쓰쏭': 33,
  '노래로': 34,
  '의지를': 35,
  '불태우자': 36,
  '일': 37,
  '다짐': 38,
  '용': 39,
  '프로필뮤직': 40,
  '지친': 41,
  '너를': 42,
  '위로해줄': 43,
  '제목편지': 44,
  '준비해봤어': 45,
  '트렌디하고': 46,
  '그루브한': 47,
  '힙합': 48,
  '알앤비': 49,
  '걍게임할때듣는음악': 50,
  '나만': 51,
  '알고싶은': 52,
  '노래들': 53,
  '축가듀엣': 54,
  '이렇게': 55,
  '비': 56,
  '내리는': 57,
  '날이면': 58,
  '너도': 59,
  '내': 60,
  '생각': 61,
  '할까': 62,
  '올림픽': 63,
  '스밍': 64,
  '목록': 65,
  '조용히': 66,
  '맥주': 67,
  '한잔': 68,
  '할때': 69,
  '새벽감성': 70,
  '명화를': 71,
  '만나는': 72,
  '시간': 73,
  '영화': 74,
  '모음': 75,
  '기분좋은': 76,
  '여름날': 77,
  '하루': 78,
  '

In [4]:
# for train, val, test
data = [train, test, val]
tag_meta = {}
tag_meta_inverse = {}
num = 0
for df in data:
    for tags in tqdm(df.tags):
        for tag in tags:
            if not tag in tag_meta:
                tag_meta[tag] = num
                tag_meta_inverse[num] = tag
                num += 1
with open('bin/tag_meta.p', 'wb') as f:
    pickle.dump((tag_meta, tag_meta_inverse), f)
display(tag_meta, tag_meta_inverse)

100%|██████████| 23015/23015 [00:00<00:00, 925033.84it/s]


{'락': 0,
 '추억': 1,
 '회상': 2,
 '까페': 3,
 '잔잔한': 4,
 '연말': 5,
 '눈오는날': 6,
 '캐럴': 7,
 '분위기': 8,
 '따듯한': 9,
 '크리스마스캐럴': 10,
 '겨울노래': 11,
 '크리스마스': 12,
 '겨울왕국': 13,
 '크리스마스송': 14,
 '댄스': 15,
 '운동': 16,
 '드라이브': 17,
 'Pop': 18,
 '트로피컬하우스': 19,
 '힐링': 20,
 '기분전환': 21,
 '2017': 22,
 '팝': 23,
 '트렌드': 24,
 '일렉': 25,
 '짝사랑': 26,
 '취향저격': 27,
 '슬픔': 28,
 '고백': 29,
 '사랑': 30,
 '이별': 31,
 '일렉트로니카': 32,
 '포크': 33,
 '메탈': 34,
 '인디': 35,
 '록': 36,
 'Metal': 37,
 '이일우': 38,
 'M에센셜': 39,
 'Rock': 40,
 'kpop': 41,
 '걸그룹댄스': 42,
 '스트레스해소': 43,
 '새해': 44,
 '여행': 45,
 '프로필음악': 46,
 '카카오톡': 47,
 '소원': 48,
 '프로필': 49,
 '소망': 50,
 '다짐': 51,
 '카톡': 52,
 '듣고': 53,
 '우울': 54,
 '이거': 55,
 '힘내': 56,
 '힙합': 57,
 '느낌있는': 58,
 '밤': 59,
 '새벽': 60,
 'RnB': 61,
 '감각적인': 62,
 '국내': 63,
 '그루브한': 64,
 '가을': 65,
 '재즈': 66,
 '감성': 67,
 '질리지않는': 68,
 '나만알고싶은': 69,
 '봄': 70,
 '설렘': 71,
 '비오는날': 72,
 '누군가생각날때': 73,
 '스밍': 74,
 '목록': 75,
 '폐막식': 76,
 '올림픽': 77,
 '엑소': 78,
 '조용히': 79,
 '혼자': 80,
 '또는': 81,
 '새벽감성': 82,
 '고민': 83,
 

{0: '락',
 1: '추억',
 2: '회상',
 3: '까페',
 4: '잔잔한',
 5: '연말',
 6: '눈오는날',
 7: '캐럴',
 8: '분위기',
 9: '따듯한',
 10: '크리스마스캐럴',
 11: '겨울노래',
 12: '크리스마스',
 13: '겨울왕국',
 14: '크리스마스송',
 15: '댄스',
 16: '운동',
 17: '드라이브',
 18: 'Pop',
 19: '트로피컬하우스',
 20: '힐링',
 21: '기분전환',
 22: '2017',
 23: '팝',
 24: '트렌드',
 25: '일렉',
 26: '짝사랑',
 27: '취향저격',
 28: '슬픔',
 29: '고백',
 30: '사랑',
 31: '이별',
 32: '일렉트로니카',
 33: '포크',
 34: '메탈',
 35: '인디',
 36: '록',
 37: 'Metal',
 38: '이일우',
 39: 'M에센셜',
 40: 'Rock',
 41: 'kpop',
 42: '걸그룹댄스',
 43: '스트레스해소',
 44: '새해',
 45: '여행',
 46: '프로필음악',
 47: '카카오톡',
 48: '소원',
 49: '프로필',
 50: '소망',
 51: '다짐',
 52: '카톡',
 53: '듣고',
 54: '우울',
 55: '이거',
 56: '힘내',
 57: '힙합',
 58: '느낌있는',
 59: '밤',
 60: '새벽',
 61: 'RnB',
 62: '감각적인',
 63: '국내',
 64: '그루브한',
 65: '가을',
 66: '재즈',
 67: '감성',
 68: '질리지않는',
 69: '나만알고싶은',
 70: '봄',
 71: '설렘',
 72: '비오는날',
 73: '누군가생각날때',
 74: '스밍',
 75: '목록',
 76: '폐막식',
 77: '올림픽',
 78: '엑소',
 79: '조용히',
 80: '혼자',
 81: '또는',
 82: '새벽감성',
 83: '고민',
 

In [15]:
# apply tag_meta
data = [train, val, test]
new_data = []
for df in data:
    new = df.copy()
    for i, tags in tqdm(enumerate(df.tags)):
        t = []
        for tag in tags:
            t.append(tag_meta[tag])
        new.at[i, 'tags'] = t
    new_data.append(new)
display(new_data[0], new_data[1], new_data[2])
with open('bin/data_tags_are_int.p', 'wb') as f:
    pickle.dump(new_data, f)

115071it [00:00, 115869.54it/s]
23015it [00:00, 135393.50it/s]
10740it [00:00, 129941.14it/s]


,id,plylst_title,songs,tags,like_cnt,updt_date
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[0],71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[1, 2]",1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[3, 4]",17,2017-08-28 07:09:34.000
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",33,2019-12-05 15:15:18.000
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[15],9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[2083, 2287, 36, 1023, 34, 0, 22994]",3,2020-04-17 04:31:11.000
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",[25],13,2015-12-24 17:23:19.000
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[9387, 455, 474, 447, 5836, 9388, 30, 1223]",4,2019-08-16 20:59:22.000
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[4, 210, 8561, 18, 5685, 187]",4,2019-10-25 23:40:42.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],1675,2019-05-27 14:14:33.000
1,131447,앨리스테이블,[],[],1,2014-07-16 15:24:24.000
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],62,2008-06-21 23:26:22.000
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],20,2017-10-30 18:15:43.000
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[4],17,2015-12-17 14:06:05.000
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[11913, 335, 3162, 455, 23086]",10,2020-04-16 21:35:44.000
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[],0,2019-03-27 15:27:40.000
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[],31,2015-11-18 11:49:09.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[],6,2012-09-29 01:57:26.000
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[477, 1021]",0,2019-12-17 14:06:45.000
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[17],21,2015-05-23 10:44:48.000
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[8],0,2019-03-14 09:47:34.000
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[],38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[1],16,2016-04-18 11:02:09.000
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[3195, 54, 8, 17, 120]",1,2020-04-08 07:15:59.000
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",[21],6,2016-06-29 00:57:21.000
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[111],4,2015-06-06 09:52:01.000


### Preprocessing Xs

In [ ]:
%%time
lst = [train, val, test]
Xs = []
for data in lst:
    api = KhaiiiApi()
    ko_nouns = {}
    ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dense=True)
    ko_nouns["title"] = tool.ko_extract_tag_from(ko_title, api, limit=None)
    ko_nouns["tag"] = tool.ko_extract_tag_from(ko_tag, api)
    ko_title, ko_tag, en_title, en_tag = tool.extract_playlist_from(data, dtype="list", dense=True)
    new_data = data[['id', 'plylst_title', 'songs']]
    new_data["tags"] = np.nan
    new_data["tags"] = new_data["tags"].astype('object')
    for i in tqdm(data.index):
        a = ko_tag[i] if i in ko_tag else []
        b = en_tag[i] if i in en_tag else []
        c = ko_nouns["title"][i] if i in ko_nouns["title"] else []
        d = ko_nouns["tag"][i] if i in ko_nouns["tag"] else []
        e = list(set(a + b + c + d))
        new_data.at[i, "tags"] = e
    Xs.append(new_data)
display(Xs[0], Xs[1], Xs[2])
with open("bin/Xs.p", 'wb') as f:
    pickle.dump(Xs, f)

### Test Code

In [10]:
api = KhaiiiApi()
def morphs(api, sentence):
    m = []
    print(sentence)
    for word in api.analyze(sentence):
        for morph in word.morphs:
            m.append((morph.lex, morph.tag))
    return m
morphs(api, '락')

락


[('락', 'MAG')]

In [20]:
s='한자韓子는 싫고, 한글은 nice하다. English 쵝오 -_-ㅋㅑㅋㅑ ./?!'
p = re.compile('[ㄱ-ㅣ가-힣]+')
print(p.findall(s))
q = re.compile('[^ ㄱ-ㅣ가-힣]+')
t = q.sub(' ', s)
t

['한자', '는', '싫고', '한글은', '하다', '쵝오', 'ㅋㅑㅋㅑ']


'한자 는 싫고  한글은  하다    쵝오  ㅋㅑㅋㅑ  '

### FastText Korean

In [10]:
%%time
ko_model = fasttext.load_facebook_model('../../fasttext/cc.ko.300.bin')

CPU times: user 1min 32s, sys: 1min 22s, total: 2min 54s
Wall time: 4min 48s


In [19]:
for w, sim in ko_model.wv.similar_by_word('여행', 50):
    print(f'{w}: {sim}')

15월의: 0.5884130001068115
코스산: 0.568044900894165
돌뿌딩이: 0.5440756678581238
자동: 0.5413090586662292
17월의: 0.5311351418495178
가이드일본: 0.5228801965713501
예약: 0.5215001106262207
코스레이크: 0.5173512697219849
旅行: 0.5167638063430786
번역: 0.5143898129463196
여행보간: 0.5067660212516785
않을: 0.5018413662910461
숙박특가: 0.49609655141830444
먹부림의: 0.4931199550628662
6츄라우미: 0.4905807375907898
경비: 0.4875657856464386
쉬멍: 0.48388007283210754
숙박코드: 0.4799029231071472
코스노스: 0.47705021500587463
푸껫: 0.4734245538711548
요리: 0.4725126326084137
될: 0.4720313549041748
보기근처: 0.47198915481567383
뜨랑: 0.4672425389289856
달세계: 0.46486955881118774
관광: 0.4630843997001648
괌: 0.46202194690704346
본가밀면: 0.4615764617919922
유형가족: 0.46120408177375793
중구신규: 0.46064674854278564
여행스타그램: 0.459503710269928
탐방: 0.4592190980911255
야영: 0.4586060345172882
가이드태국: 0.45800846815109253
신혼: 0.45747655630111694
수: 0.45694997906684875
염동집: 0.45671072602272034
青砥: 0.45598194003105164
7박: 0.4542434811592102
渋谷: 0.4522232413291931
숙박: 0.4513387978076935
코스포트: 

In [49]:
for i in train.index:
    _tags = train.tags[i]
    for j, _tag in enumerate(_tags):
        try:
            word1 = _tag
            word2 = _tags[j+1]
            print(word1, word2, ko_model.wv.similarity(word1, word2))
        except:
            pass
    if i == 3:
        break

추억 회상 0.4180501
까페 잔잔한 0.12807237
연말 눈오는날 0.25183606
눈오는날 캐럴 0.18177678
캐럴 분위기 0.114136875
분위기 따듯한 0.25572002
따듯한 크리스마스캐럴 0.26866436
크리스마스캐럴 겨울노래 0.25952664
겨울노래 크리스마스 0.27037787
크리스마스 겨울왕국 0.31709558
겨울왕국 크리스마스송 0.31680033


In [55]:
word1, word2 = '', ''
ko_model.wv.similarity(word1, word2)

0.06353715

In [53]:
for i in train.index:
    print(train.tags[i])
    if i == 10:
        break

['락']
['추억', '회상']
['까페', '잔잔한']
['연말', '눈오는날', '캐럴', '분위기', '따듯한', '크리스마스캐럴', '겨울노래', '크리스마스', '겨울왕국', '크리스마스송']
['댄스']
['운동', '드라이브', 'Pop', '트로피컬하우스', '힐링', '기분전환', '2017', '팝', '트렌드', '일렉']
['짝사랑', '취향저격', '슬픔', '고백', '사랑', '이별']
['잔잔한', '추억', '회상']
['일렉트로니카', '포크', '메탈', '락', '댄스', '인디']
['록', 'Metal', '이일우', 'M에센셜', '메탈', 'Rock', '락']
['kpop', '댄스', '걸그룹댄스', '스트레스해소']


In [64]:
for i, j in enumerate(tqdm(train.tags)):
    print(j)
    if i == 5:
        break

['락']
['추억', '회상']
['까페', '잔잔한']
['연말', '눈오는날', '캐럴', '분위기', '따듯한', '크리스마스캐럴', '겨울노래', '크리스마스', '겨울왕국', '크리스마스송']
['댄스']
['운동', '드라이브', 'Pop', '트로피컬하우스', '힐링', '기분전환', '2017', '팝', '트렌드', '일렉']

